> # `Load the Libraries`

In [19]:
import math
import matplotlib.pyplot as plt
%matplotlib
import keras
import pandas as pd
import numpy as np
from keras.models import Sequential, load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import *
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
import yfinance as yf

Using matplotlib backend: Qt5Agg


> # `Input Desired Stock Ticker and Time Period `

In [20]:
ticker = "MSFT"
period = "5y"

# get market data from Yahoo Finance
stock = yf.Ticker(ticker)
df = stock.history(period=period)
df = df.reset_index()
for i in ['Open', 'High', 'Close', 'Low']: 
      df[i]  =  df[i].astype('float64')

df.head(5)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2016-09-28,53.552246,53.718788,53.357946,53.691029,20536400,0.0,0
1,2016-09-29,53.487483,53.820562,52.932343,53.108139,25463500,0.0,0
2,2016-09-30,53.265419,53.450465,53.052617,53.293175,29910800,0.0,0
3,2016-10-03,53.117374,53.246906,52.793546,53.126625,19189500,0.0,0
4,2016-10-04,52.987850,53.293174,52.710282,52.960094,20085900,0.0,0


In [21]:
# split data into into training and test. about 2/3 to train and 1/3 to test. Want to predict closing price
training_set = df.iloc[:math.floor(len(df)*(2/3)), 1:2].values
test_set = df.iloc[math.floor(len(df)*(1/3)):, 1:2].values

> # `Model uses 60 days to predict one day`

In [22]:
# with time lag 1
# scale values to fit into 0,1
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
# 60 time-steps and 1 output
X_train = []
y_train = []
for i in range(60, len(training_set_scaled)):
    X_train.append(training_set_scaled[i-60:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))


> # `Build, Train, and Save the Model. Takes time, depending on epochs and batch size`

In [23]:
# model = Sequential()
# # first LSTM layer 
# model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], 1)))
# model.add(Dropout(0.2))
# # second LSTM layer
# model.add(LSTM(units = 50, return_sequences = True))
# model.add(Dropout(0.2))
# # third LSTM layer
# model.add(LSTM(units = 50, return_sequences = True))
# model.add(Dropout(0.2))
# # fourth LSTM layer 
# model.add(LSTM(units = 50))
# model.add(Dropout(0.2))
# # output layer
# model.add(Dense(units = 1))

# # compiling RNN
# model.compile(optimizer = 'adam', loss = 'mean_squared_error')

# # fit RNN to training data
# model.fit(X_train, y_train, epochs = 100, batch_size = 32)

# model.save('stock_prediction.h5')

In [24]:
# reshape test data
model = load_model('stock_prediction.h5')

dataset_train = df.iloc[:len(training_set_scaled), 1:2]
dataset_test = df.iloc[len(training_set_scaled):, 1:2]
dataset_total = pd.concat((dataset_train, dataset_test), axis = 0)
inputs = dataset_total[len(dataset_total) - len(dataset_test) - 60:].values
inputs = inputs.reshape(-1,1)
inputs = sc.transform(inputs)
X_test = []
for i in range(60, len(inputs)):
    X_test.append(inputs[i-60:i, 0])
X_test = np.array(X_test)
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))


In [25]:
# make predictions using the test data
predicted_stock_price = model.predict(X_test)
predicted_stock_price = sc.inverse_transform(predicted_stock_price)

> # `Prediction for the Next Day`

In [26]:
# forecast next day
actual = [inputs[len(inputs) + 1 - 60:len(inputs)]]
actual = np.array(actual)
actual = np.reshape(actual, (actual.shape[0], actual.shape[1],1))

forecast = model.predict(actual)
forecast = sc.inverse_transform(forecast)

current_price = df["Close"].iloc[-1]
print(f"The current price of {ticker} is: {current_price}")
print(f"The predicted price of {ticker} for next day is: {float(forecast)}")

The current price of MSFT is: 294.1700134277344
The predicted price of MSFT for next day is: 284.3259582519531


> # `Opens the plot in a new window`

In [27]:
# Visualising the results
plt.plot(df.loc[len(training_set_scaled):, "Date"],dataset_test.values, color = "red", label = "Real Stock Price")
plt.plot(df.loc[len(training_set_scaled):, "Date"],predicted_stock_price, color = "cyan", label = "Predicted Stock Price")
plt.title(f"{ticker} Stock Price Prediction")
plt.xlabel('Date')
plt.ylabel('Stock Price')
plt.legend()
plt.show()